In [1]:
import numpy as np
import pandas as pd
from scipy import stats 
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import re

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/manipulaeHealth/desafio-machine-learning/main/dados_preco.csv")
df.head()

,descricao,criado,qtdInsumos,calculado,correto
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0


In [3]:
df.describe()

,qtdInsumos,calculado,correto
count,7121.000000,7121.000000,7121.000000
mean,3.281842,137.089552,135.891798
std,3.194301,191.266516,189.806974
min,1.000000,30.000100,33.000000
25%,1.000000,49.786100,49.500000
50%,2.000000,87.350300,83.100000
75%,4.000000,165.167000,161.200000
max,26.000000,10364.992500,10356.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7121 entries, 0 to 7120
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   descricao   7121 non-null   object 
 1   criado      7121 non-null   object 
 2   qtdInsumos  7121 non-null   int64  
 3   calculado   7121 non-null   float64
 4   correto     7121 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 278.3+ KB


In [5]:
def tabelaresumo(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Nome'] = summary['index']
    summary = summary[['Nome','dtypes']]
    summary['Valores Faltantes'] = df.isnull().sum().values    
    summary['Valores Únicos'] = df.nunique().values
    summary['Primeiro Valor'] = df.loc[0].values
    summary['Segundo Valor'] = df.loc[1].values
    summary['Terceiro Valor'] = df.loc[2].values

    for name in summary['Nome'].value_counts().index:
        summary.loc[summary['Nome'] == name, 'Entropia'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary

In [6]:
tabelaresumo(df)

Dataset Shape: (7121, 5)


,Nome,dtypes,Valores Faltantes,Valores Únicos,Primeiro Valor,Segundo Valor,Terceiro Valor,Entropia
0,descricao,object,0,7121,30 CAP | BUPROPIONA CLORIDRATO 150MG,60 CAP | FINASTERIDA 1:10 1MG,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,12.80
1,criado,object,0,5163,2020-08-05 23:19:09,2020-08-05 22:51:09,2020-08-05 22:51:09,12.15
2,qtdInsumos,int64,0,23,1,1,3,2.86
3,calculado,float64,0,5730,47.5297,46.962,105.139,12.31
4,correto,float64,0,2308,39.9,43.6,111.4,9.59


In [7]:
lst = []
qtdd_cap = []
for descricao in df.descricao:
    lst = descricao.split(" ")
    qtdd_cap.append(lst[0])

df["quantidade (capsulas)"] = qtdd_cap
df.head()

,descricao,criado,qtdInsumos,calculado,correto,quantidade (capsulas)
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9,30
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6,60
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4,60
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6,60
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0,120


In [8]:
lst = []
s = []
for descricao in df.descricao:
    lst = descricao.split("| ")
    s.append(lst[1])

df["formula"] = s
df.head()

,descricao,criado,qtdInsumos,calculado,correto,quantidade (capsulas),formula
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9,30,BUPROPIONA CLORIDRATO 150MG
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6,60,FINASTERIDA 1:10 1MG
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4,60,NAC 250MG; SILIMARINA 150MG; SAME 50MG
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6,60,ANASTROZOL 1:10 0.300MG
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0,120,DUTASTERIDA 0.250MG; TADALAFIL 10MG


In [9]:
lst = []
s = []
for i in df.formula:
    lst = i.split("; ")
    s.append(lst[0])

df['insumo 1'] = s
df.head()

,descricao,criado,qtdInsumos,calculado,correto,quantidade (capsulas),formula,insumo 1
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9,30,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO 150MG
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6,60,FINASTERIDA 1:10 1MG,FINASTERIDA 1:10 1MG
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4,60,NAC 250MG; SILIMARINA 150MG; SAME 50MG,NAC 250MG
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6,60,ANASTROZOL 1:10 0.300MG,ANASTROZOL 1:10 0.300MG
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0,120,DUTASTERIDA 0.250MG; TADALAFIL 10MG,DUTASTERIDA 0.250MG


In [10]:
df['insumo_1']=df['insumo 1'].str.extract(r'(\D*\s?\D*)', expand = True)
df.head(20)

,descricao,criado,qtdInsumos,calculado,correto,quantidade (capsulas),formula,insumo 1,insumo_1
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9,30,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6,60,FINASTERIDA 1:10 1MG,FINASTERIDA 1:10 1MG,FINASTERIDA
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4,60,NAC 250MG; SILIMARINA 150MG; SAME 50MG,NAC 250MG,NAC
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6,60,ANASTROZOL 1:10 0.300MG,ANASTROZOL 1:10 0.300MG,ANASTROZOL
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0,120,DUTASTERIDA 0.250MG; TADALAFIL 10MG,DUTASTERIDA 0.250MG,DUTASTERIDA
5,240 CAP | TRIIODOTIRONINA SLOW RELEASE 12MCG,2020-08-05 22:48:09,1,40.6362,77.0,240,TRIIODOTIRONINA SLOW RELEASE 12MCG,TRIIODOTIRONINA SLOW RELEASE 12MCG,TRIIODOTIRONINA SLOW RELEASE
6,120 CAP | T4 60MCG,2020-08-05 22:47:09,1,35.1005,50.0,120,T4 60MCG,T4 60MCG,T
7,60 CAP | VITAMINA B2 30MG; VITAMINA B3 60MG;...,2020-08-05 22:40:09,14,248.2063,272.2,60,VITAMINA B2 30MG; VITAMINA B3 60MG; VITAMINA...,VITAMINA B2 30MG,VITAMINA B
8,60 CAP | CORTISOL 20MG; PRASTERONA 20MG,2020-08-05 22:37:10,2,220.1472,206.6,60,CORTISOL 20MG; PRASTERONA 20MG,CORTISOL 20MG,CORTISOL
9,30 CAP | TADALAFIL 10MG; ARGININA 2G,2020-08-05 22:37:10,2,134.5175,127.4,30,TADALAFIL 10MG; ARGININA 2G,TADALAFIL 10MG,TADALAFIL


In [11]:
df['qtdd_1']=df['insumo 1'].str.extract(r'(\d.?\d?\d?\d?[G-M]?[A-Z][G-M])', expand = True)
df.head(15)

,descricao,criado,qtdInsumos,calculado,correto,quantidade (capsulas),formula,insumo 1,insumo_1,qtdd_1
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9,30,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO,150MG
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6,60,FINASTERIDA 1:10 1MG,FINASTERIDA 1:10 1MG,FINASTERIDA,1MG
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4,60,NAC 250MG; SILIMARINA 150MG; SAME 50MG,NAC 250MG,NAC,250MG
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6,60,ANASTROZOL 1:10 0.300MG,ANASTROZOL 1:10 0.300MG,ANASTROZOL,0.300MG
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0,120,DUTASTERIDA 0.250MG; TADALAFIL 10MG,DUTASTERIDA 0.250MG,DUTASTERIDA,0.250MG
5,240 CAP | TRIIODOTIRONINA SLOW RELEASE 12MCG,2020-08-05 22:48:09,1,40.6362,77.0,240,TRIIODOTIRONINA SLOW RELEASE 12MCG,TRIIODOTIRONINA SLOW RELEASE 12MCG,TRIIODOTIRONINA SLOW RELEASE,12MCG
6,120 CAP | T4 60MCG,2020-08-05 22:47:09,1,35.1005,50.0,120,T4 60MCG,T4 60MCG,T,60MCG
7,60 CAP | VITAMINA B2 30MG; VITAMINA B3 60MG;...,2020-08-05 22:40:09,14,248.2063,272.2,60,VITAMINA B2 30MG; VITAMINA B3 60MG; VITAMINA...,VITAMINA B2 30MG,VITAMINA B,30MG
8,60 CAP | CORTISOL 20MG; PRASTERONA 20MG,2020-08-05 22:37:10,2,220.1472,206.6,60,CORTISOL 20MG; PRASTERONA 20MG,CORTISOL 20MG,CORTISOL,20MG
9,30 CAP | TADALAFIL 10MG; ARGININA 2G,2020-08-05 22:37:10,2,134.5175,127.4,30,TADALAFIL 10MG; ARGININA 2G,TADALAFIL 10MG,TADALAFIL,10MG


In [12]:
df['qtdd_1_unit']=df['qtdd_1'].str.extract(r'([G-M]?[A-Z][G-M])', expand = True)
df.head(15)

,descricao,criado,qtdInsumos,calculado,correto,quantidade (capsulas),formula,insumo 1,insumo_1,qtdd_1,qtdd_1_unit
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9,30,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO,150MG,MG
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6,60,FINASTERIDA 1:10 1MG,FINASTERIDA 1:10 1MG,FINASTERIDA,1MG,MG
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4,60,NAC 250MG; SILIMARINA 150MG; SAME 50MG,NAC 250MG,NAC,250MG,MG
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6,60,ANASTROZOL 1:10 0.300MG,ANASTROZOL 1:10 0.300MG,ANASTROZOL,0.300MG,MG
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0,120,DUTASTERIDA 0.250MG; TADALAFIL 10MG,DUTASTERIDA 0.250MG,DUTASTERIDA,0.250MG,MG
5,240 CAP | TRIIODOTIRONINA SLOW RELEASE 12MCG,2020-08-05 22:48:09,1,40.6362,77.0,240,TRIIODOTIRONINA SLOW RELEASE 12MCG,TRIIODOTIRONINA SLOW RELEASE 12MCG,TRIIODOTIRONINA SLOW RELEASE,12MCG,MCG
6,120 CAP | T4 60MCG,2020-08-05 22:47:09,1,35.1005,50.0,120,T4 60MCG,T4 60MCG,T,60MCG,MCG
7,60 CAP | VITAMINA B2 30MG; VITAMINA B3 60MG;...,2020-08-05 22:40:09,14,248.2063,272.2,60,VITAMINA B2 30MG; VITAMINA B3 60MG; VITAMINA...,VITAMINA B2 30MG,VITAMINA B,30MG,MG
8,60 CAP | CORTISOL 20MG; PRASTERONA 20MG,2020-08-05 22:37:10,2,220.1472,206.6,60,CORTISOL 20MG; PRASTERONA 20MG,CORTISOL 20MG,CORTISOL,20MG,MG
9,30 CAP | TADALAFIL 10MG; ARGININA 2G,2020-08-05 22:37:10,2,134.5175,127.4,30,TADALAFIL 10MG; ARGININA 2G,TADALAFIL 10MG,TADALAFIL,10MG,MG


In [18]:
df['qtdd_1_final']=df['qtdd_1'].str.extract(r'(\d.?\d?\d?\d?)' , expand = True)
df['qtdd_1_final']=df['qtdd_1_final'].replace({'M':''}, regex=True)
df['qtdd_1_final']=df['qtdd_1_final'].replace({'F':''}, regex=True)
df['qtdd_1_final']=df['qtdd_1_final'].replace({'B':''}, regex=True)
df['qtdd_1_final']=df['qtdd_1_final'].replace({'\(':''}, regex=True)
df['qtdd_1_final']=df['qtdd_1_final'].replace({'-':''}, regex=True)
df['qtdd_1_final']=df['qtdd_1_final'].replace({' ':''}, regex=True)
df.head(15)

,descricao,criado,qtdInsumos,calculado,correto,quantidade (capsulas),formula,insumo 1,insumo_1,qtdd_1,qtdd_1_unit,qtdd_1_final
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9,30,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO,150MG,MG,150
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6,60,FINASTERIDA 1:10 1MG,FINASTERIDA 1:10 1MG,FINASTERIDA,1MG,MG,1
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4,60,NAC 250MG; SILIMARINA 150MG; SAME 50MG,NAC 250MG,NAC,250MG,MG,250
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6,60,ANASTROZOL 1:10 0.300MG,ANASTROZOL 1:10 0.300MG,ANASTROZOL,0.300MG,MG,0.300
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0,120,DUTASTERIDA 0.250MG; TADALAFIL 10MG,DUTASTERIDA 0.250MG,DUTASTERIDA,0.250MG,MG,0.250
5,240 CAP | TRIIODOTIRONINA SLOW RELEASE 12MCG,2020-08-05 22:48:09,1,40.6362,77.0,240,TRIIODOTIRONINA SLOW RELEASE 12MCG,TRIIODOTIRONINA SLOW RELEASE 12MCG,TRIIODOTIRONINA SLOW RELEASE,12MCG,MCG,12
6,120 CAP | T4 60MCG,2020-08-05 22:47:09,1,35.1005,50.0,120,T4 60MCG,T4 60MCG,T,60MCG,MCG,60
7,60 CAP | VITAMINA B2 30MG; VITAMINA B3 60MG;...,2020-08-05 22:40:09,14,248.2063,272.2,60,VITAMINA B2 30MG; VITAMINA B3 60MG; VITAMINA...,VITAMINA B2 30MG,VITAMINA B,30MG,MG,30
8,60 CAP | CORTISOL 20MG; PRASTERONA 20MG,2020-08-05 22:37:10,2,220.1472,206.6,60,CORTISOL 20MG; PRASTERONA 20MG,CORTISOL 20MG,CORTISOL,20MG,MG,20
9,30 CAP | TADALAFIL 10MG; ARGININA 2G,2020-08-05 22:37:10,2,134.5175,127.4,30,TADALAFIL 10MG; ARGININA 2G,TADALAFIL 10MG,TADALAFIL,10MG,MG,10


In [19]:
df["qtdd_1_final"].unique()

array(['150', '1', '250', '0.300', '0.250', '12', '60', '30', '20', '10',
       '2', '200', '10000', '100', '50', '4000', '15', '400', '300',
       '120', '500', '0.000', '3', nan, '25', '600', '40', '8', '5',
       '50000', '125', '6', '0.500', '700', '5000', '6000', '520', '90',
       '35', '70', '1000', '550', '2.500', '75', '7000', '7.500', '80',
       '350', '3.500', '60000', '1.700', '2000', '4', '66', '35000',
       '0.200', '3000', '15000', '20000', '130', '852', '4.500', '70000',
       '42', '7', '160', '9', '16', '5.000', '1.800', '25000', '210',
       '0.750', '4500', '65000', '14000', '1080', '2.400', '0.400', '45',
       '1.500', '22', '280', '220', '0', '8000', '850', '800', '1500',
       '825', '17', '1200', '38', '375', '450', '192', '750', '260',
       '00000', '180', '40000', '2500', '90000', '18', '240', '6.250',
       '540', '178', '155', '1.200', '30000', '525', '12000', '320', '98',
       '14', '1400', '0.125', '11', '80000', '48', '1.250', '1.660', '

In [20]:
df["qtdd_1_final"] = df["qtdd_1_final"].astype('float')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7121 entries, 0 to 7120
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   descricao              7121 non-null   object 
 1   criado                 7121 non-null   object 
 2   qtdInsumos             7121 non-null   int64  
 3   calculado              7121 non-null   float64
 4   correto                7121 non-null   float64
 5   quantidade (capsulas)  7121 non-null   object 
 6   formula                7121 non-null   object 
 7   insumo 1               7121 non-null   object 
 8   insumo_1               7121 non-null   object 
 9   qtdd_1                 6970 non-null   object 
 10  qtdd_1_unit            6970 non-null   object 
 11  qtdd_1_final           6970 non-null   float64
dtypes: float64(3), int64(1), object(8)
memory usage: 667.7+ KB


In [17]:
df["qtdd_1_final"].head(20)

0       150.00
1         1.00
2       250.00
3         0.30
4         0.25
5        12.00
6        60.00
7        30.00
8        20.00
9        10.00
10        2.00
11      200.00
12      200.00
13    10000.00
14       20.00
15      150.00
16       10.00
17      100.00
18       50.00
19      100.00
Name: qtdd_1_final, dtype: float64